In [1]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from konlpy.tag import Okt
from collections import defaultdict
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, RegexpStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from tqdm import tqdm
from collections import Counter

In [4]:
with open('eng_review.pk', 'rb') as f:
    eng_review = pickle.load(f)

In [21]:
with open('korea_review.pk', 'rb') as f:
    kor_review = pickle.load(f)

In [10]:
game_list = eng_review['game'].unique()

In [100]:
game = (eng_review.loc[eng_review['game'] == 'PUBG: BATTLEGROUNDS'])

In [10]:
total_game = pickle.load( open('total_game', 'rb'))

In [118]:
pickle.dump(total_game, open('total_game', 'wb'))

In [11]:
eng_review = eng_review[eng_review['game'].isin(total_game)].reset_index(drop=True)

In [109]:
game

,game,review,label
675,PUBG: BATTLEGROUNDS,"the new privacy policy is insane, it allows th...",0
676,PUBG: BATTLEGROUNDS,"its gonna be f2p from january 12 2022, dont bu...",1
677,PUBG: BATTLEGROUNDS,. you drop 2. loot for 15 minutes 3. get insan...,1
678,PUBG: BATTLEGROUNDS,"i've always asked myself, how can people play ...",0
679,PUBG: BATTLEGROUNDS,pubg is the type of game where it starts off w...,1
...,...,...,...
1638,PUBG: BATTLEGROUNDS,i just uninstalled this game. now hold the pot...,1
1639,PUBG: BATTLEGROUNDS,i really wanted to give this game a chance aga...,0
1640,PUBG: BATTLEGROUNDS,it's an ok battle royale game. for people who ...,0
1641,PUBG: BATTLEGROUNDS,"broken/flawed game, useless devs pubg is funda...",0


In [18]:
pos_neg = []
for game in total_game:
    game = (eng_review.loc[eng_review['game'] == game])
    pos, neg = pos_neg_split(game)
    value = len(pos)/(len(pos) + len(neg)) *100
    pos_neg.append(value)

In [22]:
pos_neg_kor = []
for game in total_game:
    game = (kor_review.loc[kor_review['game'] == game])
    pos, neg = pos_neg_split(game)
    value = len(pos)/(len(pos) + len(neg)) *100
    pos_neg_kor.append(value)

In [20]:
sum(pos_neg) / len(pos_neg)

81.9394638004097

In [23]:
sum(pos_neg_kor) / len(pos_neg_kor)

82.27589773033455

In [53]:
pos_eng = []
neg_eng = []
for game_name in total_game:
    game = (eng_review.loc[eng_review['game'] == game_name])
    pos, neg = pos_neg_split(game)
    value = len(pos)/(len(pos) + len(neg)) *100
    if value >= 70:
        pos_eng.append(game_name)
    else:
        neg_eng.append(game_name)

In [54]:
pos_kor = []
neg_kor = []
for game_name in total_game:
    game = (kor_review.loc[kor_review['game'] == game_name])
    pos, neg = pos_neg_split(game)
    value = len(pos)/(len(pos) + len(neg)) *100
    if value >= 70:
        pos_kor.append(game_name)
    else:
        neg_kor.append(game_name)

In [55]:
len(set(pos_eng) & set(pos_kor))

260

In [56]:
set(pos_eng) & set(pos_kor)

{'100% Orange Juice',
 '20 Minutes Till Dawn',
 '60 Seconds!',
 '911 Operator',
 'A Dance of Fire and Ice',
 'A Way Out',
 'ABZU',
 'ACE COMBAT™ 7: SKIES UNKNOWN',
 'ALTF4',
 'ASTRONEER',
 'Action Taimanin',
 'Alice: Madness Returns',
 'Among Us',
 'Angels of Death',
 'Arma 3',
 "Assassin's Creed® Odyssey",
 "Assassin's Creed® Origins",
 'Assetto Corsa',
 'Assetto Corsa Competizione',
 'Attack on Titan 2 - A.O.T.2',
 'Baba Is You',
 'Bad North: Jotunn Edition',
 'Banished',
 'Bastion',
 'Battlefield 1 ™',
 'Battlefield 4™',
 'Bayonetta',
 'Black Mesa',
 'Bloodstained: Ritual of the Night',
 'Bloons TD 6',
 'Borderlands 2',
 'Borderlands Game of the Year',
 'Braid',
 'CARRION',
 'CHRONO TRIGGER®',
 'Call of Duty®: Modern Warfare® 2 (2009)',
 'Celeste',
 'Changed',
 'Children of Morta',
 'Cities: Skylines',
 'Clone Drone in the Danger Zone',
 'Command & Conquer™ Remastered Collection',
 'Company of Heroes',
 'Control Ultimate Edition',
 'Core Keeper',
 'Counter-Strike: Global Offensive',

In [57]:
len(set(pos_eng) & set(neg_kor))

11

In [58]:
set(pos_eng) & set(neg_kor)

{'Borderlands: The Pre-Sequel',
 'Company of Heroes 2',
 'Demon Slayer -Kimetsu no Yaiba- The Hinokami Chronicles',
 'Eternal Return',
 'FINAL FANTASY® XIII',
 'Gang Beasts',
 'One Piece Pirate Warriors 3',
 'Portal Knights',
 'Ruined King: A League of Legends Story™',
 'SD GUNDAM BATTLE ALLIANCE',
 'Zelter'}

In [59]:
len(set(neg_eng) & set(pos_kor))

31

In [60]:
set(neg_eng) & set(pos_kor)

{'ARK: Survival Evolved',
 'Albion Online',
 'Apex Legends™',
 'Borderlands 3',
 'Bro Falls',
 'DARK SOULS™: REMASTERED',
 'Dragon Age™ Inquisition',
 'EVE Online',
 'Fallout 4',
 'Forager',
 'Graveyard Keeper',
 'Jurassic World Evolution',
 'Microsoft Flight Simulator Game of the Year Edition',
 'NARAKA: BLADEPOINT',
 'Red Dead Online',
 'Sid Meier’s Civilization® VI',
 'State of Decay 2: Juggernaut Edition',
 'Subnautica: Below Zero',
 'The Isle',
 'The Sims™ 4',
 'They Are Billions',
 'Tom Clancy’s The Division™',
 'Total War: WARHAMMER',
 'Total War™: ROME II - Emperor Edition',
 'Tropico 6',
 'Unpacking',
 'War Thunder',
 'Wasteland 3',
 'World of Tanks',
 'World of Warships',
 'Yu-Gi-Oh! Duel Links'}

In [61]:
len(set(neg_eng) & set(neg_kor))

42

In [62]:
set(neg_eng) & set(neg_kor)

{'Age of Empires III: Definitive Edition',
 "Assassin's Creed® Unity",
 'Battlefield™ 2042',
 'Black Squad',
 'Bless Unleashed',
 'Bus Simulator 18',
 'CROWZ',
 'Call of Duty®: Black Ops III',
 'Call of Duty®: Infinite Warfare',
 'Call of Duty®: Modern Warfare® Remastered',
 'Call of Duty®: WWII',
 "Conqueror's Blade",
 'Cyberpunk 2077',
 'DEAD OR ALIVE 6',
 'DayZ',
 'Dead by Daylight',
 'Destiny 2',
 'FIFA 22',
 'Fallout 76: The Pitt',
 'Grand Theft Auto V',
 'GrandChase',
 'Hero Siege',
 'ICARUS',
 'Mount & Blade II: Bannerlord',
 'New World',
 "No Man's Sky",
 'PUBG: BATTLEGROUNDS',
 'PixARK',
 'Poppy Playtime',
 'Propnight',
 'ROMANCE OF THE THREE KINGDOMS XIV',
 'SCUM',
 "Sid Meier's Civilization®: Beyond Earth™",
 'Street Fighter V',
 'Super Mecha Champions',
 'The Crew™ 2',
 'The Cycle: Frontier',
 'Total War: THREE KINGDOMS',
 'Wolcen: Lords of Mayhem',
 'World of Tanks Blitz',
 'Yu-Gi-Oh! Master Duel',
 'eFootball™ 2023'}

In [101]:
def pos_neg_split(game):
    pos_review = []
    neg_review = []
    game = game.reset_index(drop =True)
    for i in range(len(game)):
        if game['label'][i] == 0:
            neg_review += [game['review'][i]]
        elif game['label'][i] == 1:
            pos_review += [game['review'][i]]
    return pos_review, neg_review

In [102]:
pos, neg = pos_neg_split(game)

In [103]:
len(pos), len(neg)

(376, 592)

In [104]:
len(pos)/(len(pos) + len(neg)) *100

38.84297520661157

In [105]:
sum_sen = pos + neg

In [281]:
# 75% 긍정적
# under 부정

In [80]:
game_stop_word = ['play', 'game', 'played', 'steam', 'playing', 'the', 'and', 'to', 'it', 'is', 'of', 'in', 'this', 'you',
 'but', 'for', 'that', 'with', 'on', 'so', 'they', 'are', 'at', 'me', 'from', 'just', 'if', 'by', 'even', 'still',
 'your', 'be', 'my', 'or', 'more', 'there', 'no', 'not', 'their', 'good', 'like', 'can', 'than', 'after', 'do', 'because',
 'ha', 'wa', 'really', 'one', 'when', 'will','an','about', 'out', 'who', 'don', 'have', 'them', 'what', 'into', 'fun',
 'get', 'up', 'we', 'other', 'been', 'some', 'only', 'now', 'would', 'want', 'every', 'all', 'many', 'over', 'then',
 'had', 'make', 'any', 'most', 'how', 'much', 'very', 'lot', 'which', 'also', 'too', 'well', 'better', 'new', 'go',
 'thing', 'where', 'hasn', 'he', "shan't", "haven't", "hadn't", 'am', 'further', 'ours', 'won', 'until', 'same', "doesn't",
 "you've", 'his', 'during', "weren't", "wasn't", "that'll", 'o', 'our', 'through', 'its', "hasn't", 'was', 'aren', "it's",
 'were', 'again', 'few', 'itself', 'doesn', 'both', 'needn', 'being', 'under', 'll', "she's", "shouldn't", 'doing',
 'ma', "aren't", 'against', 're', 'mustn', "don't", 'haven', 'isn', 'having', 'below', 'ourselves', 'myself', 'off',
 'weren', 'wasn', 'why', 'i', 'does', 'whom', 'those', 'has', 'themselves', 'hers', 'such', 's', "you're", 'ain',
 "couldn't", 'shouldn', 'own', 'couldn', 'm', 'her', 'each', 'wouldn', 'him', 'herself', 'yourselves', 'mightn',
 'down', 'here', 'didn', 'these', 'above', 'once', "wouldn't", 'hadn', 't', 'while', "won't", 'a', 'before', "didn't",
 "needn't", "should've", "mustn't", 'as', 'theirs', 've', 'did', "you'd", "mightn't", "you'll", 'nor', 'himself',
 'between', 'y', 'yours', 'd', "isn't", 'shan', 'she', 'yourself', 'should', 'gameplay', 'ingame',]


In [132]:
def regular_expression(review):
    
    num = 0
    while num < len(review):
        description = review[num]
        #description = re.sub('[^a-z]+',' ',description)
        #review[num] = re.sub('[-]{2,}',' ',description)
        review[num] = re.sub('[^a-z]+',' ',description)
        num += 1
        
    return review

def tokenizing(review):
    
    tokenized = []
    for sen in review:
        tokenized.append(word_tokenize(sen))

    return tokenized

def normalize_data(tokenized, normalization):
    
    # input string lower
    normalization = normalization.lower()
    
    # porterstemmer
    if 'porter' in normalization:
        
        porter_list = []
        porter = PorterStemmer()

        for sen in tokenized:
            second_list = []
            for i in sen:
                second_list.append(porter.stem(i))
            porter_list.append(second_list)
            
        return porter_list
    
    # lemmatizer
    if 'lemma' in normalization:
        
        lemmatizer_list = []
        lemmatizer = WordNetLemmatizer()

        for sen in tokenized:
            second_list = []
            for i in sen:
                second_list.append(lemmatizer.lemmatize(i))
            lemmatizer_list.append(second_list)
        return lemmatizer_list
    
def stop_words(data):
    
    stop_word = game_stop_word
    result = []
    for i in range(len(data)):
        temp = []
        for word in data[i]:
                if word not in stop_word:
                    temp.append(word)
        result.append(temp)
        
    return result

def extract_nouns(tokens):
    result = []
    for token in tokens:
        tagged = nltk.pos_tag(token)
        allnoun = [word for word, pos in tagged if pos in ['NN','NNP']]
        result += [allnoun]
    return result
    
def preprocessing(data, normalization):
    
    review = regular_expression(data)
    
    tokenized = tokenizing(review)
    
    normalized = normalize_data(tokenized, normalization)
    
    result = stop_words(normalized)

    result = extract_nouns(result)
    
    return result

In [115]:
neg = neg[:5]

In [116]:
preprocessing(neg, 'lemma')

[['the', 'new', 'privacy', 'policy', 'is', 'insane', 'it', 'allows', 'them', 'to', 'acess', 'and', 'store', 'and', 'share', 'ridiculous', 'amounts', 'of', 'information', 'that', 'they', 'have', 'no', 'buisiness', 'knowing', 'i', 'have', 'puchased', 'this', 'game', 'years', 'ago', 'and', 'now', 'they', 'block', 'me', 'from', 'playing', 'it', 'unless', 'i', 'forefeit', 'my', 'right', 'to', 'privacy', 'i', 'consider', 'this', 'to', 'be', 'fraud', 'edit', 'here', 'are', 'some', 'examples', 'information', 'that', 'they', 'collect', 'includes', 'nickname', 'phone', 'number', 'steamid', 'purchase', 'history', 'etc', 'to', 'provide', 'and', 'operate', 'the', 'game', 'nickname', 'email', 'address', 'device', 'information', 'and', 'records', 'of', 'the', 'issues', 'you', 'experience', 'to', 'provide', 'customer', 'service', 'and', 'and', 'information', 'for', 'delivering', 'goods', 'i', 'e', 'and', 'for', 'participation', 'in', 'the', 'survey', 'and', 'other', 'events', 'when', 'you', 'participa

[['privacy',
  'policy',
  'insane',
  'ace',
  'store',
  'share',
  'amount',
  'information',
  'buisiness',
  'knowing',
  'year',
  'block',
  'forefeit',
  'right',
  'privacy',
  'fraud',
  'edit',
  'example',
  'information',
  'collect',
  'phone',
  'number',
  'purchase',
  'history',
  'operate',
  'email',
  'address',
  'device',
  'information',
  'record',
  'issue',
  'experience',
  'customer',
  'service',
  'information',
  'participation',
  'survey',
  'event',
  'feature',
  'information',
  'information',
  'user',
  'consent',
  'name',
  'reason',
  'information',
  'explanation',
  'reader',
  'information',
  'collect',
  'use',
  'information',
  'come',
  'share',
  'information',
  'share',
  'company',
  'service',
  'player',
  'behavior',
  'activity',
  'activity',
  'player',
  'consent',
  'read',
  'share',
  'information',
  'case',
  'understand'],
 ['hour',
  'thumb',
  'server',
  'tatter',
  'log',
  'report',
  'session',
  'nothing',
  'cha

In [153]:
dd = preprocessing(sum_sen, 'lemma')

In [17]:
dd = stop_words(dd)

In [112]:
eng_review = eng_review[eng_review['game'].isin(total_game)].reset_index(drop=True)

In [113]:
eng_review2 = eng_review.groupby(['game', 'label']).review.apply(list)

In [173]:
eng_review2

game                                                                label
100% Orange Juice                                                   0        [warning: toxic community, unprofessional and ...
                                                                    1        [0% orange juice is an adorable game that succ...
20 Minutes Till Dawn                                                0        [i really wanna like this game but, due to the...
                                                                    1        [you need to deal 22 500 dmg to destroy the tr...
60 Seconds!                                                         0        [0 seconds is a very standard resource managem...
                                                                    1        [0 seconds is a hilarious game in which you ha...
911 Operator                                                        0        [if you're too good at placing your units arou...
                                     

In [ ]:
game_list = eng_review['game'].unique()

In [182]:
eng_review2.index[2][0]

'20 Minutes Till Dawn'

In [207]:
def split_review(review_lang, total_game):
    
    review_lang = review_lang[review_lang['game'].isin(total_game)].reset_index(drop=True)
    review_lang = review_lang.groupby(['game', 'label']).review.apply(list)
    game_list = []
    for i in tqdm(range(len(review_lang))):
        if i % 2 == 0:
            
            #df = pd.DataFrame({'game_name'}, {'negative'}, {'positive'})
            #df['game_name'] = game_list[num]
            dict_game = {}
            neg = preprocessing(review_lang[i], 'lemma')
            neg = [j for i in neg for j in set(i)]
            neg = [i for i in neg if len(i) > 2]
            
            
        if i % 2 == 1:
            pos = preprocessing(review_lang[i], 'lemma')
            pos = [j for i in pos for j in set(i)]
            pos = [i for i in pos if len(i) > 2]
            game_name = review_lang.index[i][0]
            dict_game[game_name] = neg, pos
            game_list.append(dict_game)
            # pickle.dump(dict_game, open('./data/english/'+ game_name, 'wb'))
    return game_list
            

In [208]:
game_list = split_review(eng_review, total_game)

100%|██████████| 688/688 [12:16<00:00,  1.07s/it]


In [262]:
game_list[0]

{'100% Orange Juice': (['market',
   'file',
   'friend',
   'side',
   'fake',
   'moderation',
   'bounty',
   'effect',
   'qhgrw',
   'lash',
   'drama',
   'trigger',
   'issue',
   'work',
   'retention',
   'pet',
   'power',
   'victim',
   'card',
   'function',
   'style',
   'staff',
   'design',
   'amount',
   'rng',
   'com',
   'price',
   'matter',
   'behaviour',
   'hunt',
   'balance',
   'check',
   'school',
   'dot',
   'stuff',
   'item',
   'grinder',
   'system',
   'forum',
   'name',
   'estimate',
   'doe',
   'quality',
   'bloat',
   'poppo',
   'ozl',
   'kid',
   'png',
   'case',
   'sign',
   'community',
   'playtime',
   'anyone',
   'mio',
   'orange',
   'sora',
   'toxic',
   'life',
   'stranger',
   'money',
   'hidden',
   'gacha',
   'txt',
   'version',
   'grind',
   'dislike',
   'truth',
   'character',
   'evidence',
   'trading',
   'board',
   'hell',
   'tldr',
   'remain',
   'focus',
   'plain',
   'criticism',
   'misuse',
   'refus

In [255]:
with open('eng_game_word.pk', 'wb') as f:
    pickle.dump(game_list,f)

In [3]:
with open('eng_game_word.pk', 'rb') as f:
    game_list = pickle.load(f)

In [4]:
find_name = input()
for i in range(len(game_list)):
    if find_name in list(game_list[i].keys())[0].lower():
        print(i, list(game_list[i].keys())[0])

elden
106 ELDEN RING


In [6]:
index = 212

In [145]:
neg_word = list(game_list[index].values())[0][0] # 부정
pos_word = list(game_list[index].values())[0][1] # 긍정
neg_pos = neg_word + pos_word

In [146]:
count = Counter(neg_word)
word_list = count.most_common(100)

In [147]:
word_list

[('server', 60),
 ('player', 39),
 ('time', 35),
 ('season', 32),
 ('money', 24),
 ('issue', 23),
 ('respawn', 23),
 ('bug', 22),
 ('match', 19),
 ('system', 19),
 ('level', 18),
 ('problem', 18),
 ('fix', 18),
 ('apex', 17),
 ('hour', 17),
 ('skin', 17),
 ('lobby', 16),
 ('day', 15),
 ('battle', 15),
 ('cheater', 15),
 ('character', 15),
 ('year', 14),
 ('reason', 14),
 ('legend', 13),
 ('week', 13),
 ('team', 13),
 ('devs', 13),
 ('nothing', 12),
 ('map', 12),
 ('way', 12),
 ('anything', 12),
 ('movement', 12),
 ('care', 12),
 ('month', 11),
 ('account', 11),
 ('work', 11),
 ('skill', 11),
 ('origin', 11),
 ('something', 10),
 ('experience', 10),
 ('ability', 10),
 ('crash', 10),
 ('everyone', 10),
 ('everything', 10),
 ('support', 9),
 ('chance', 9),
 ('update', 9),
 ('event', 9),
 ('heirloom', 9),
 ('life', 9),
 ('shooter', 9),
 ('teammate', 9),
 ('friend', 8),
 ('someone', 8),
 ('dollar', 8),
 ('trash', 8),
 ('sbmm', 8),
 ('matchmaking', 8),
 ('pas', 8),
 ('sound', 8),
 ('cheat', 

In [196]:
saved_game = pickle.load( open('./data/english/'+ 'ICARUS', 'rb'))

In [100]:
dict_eng = dict()
for game_name, game_review in tqdm(zip(eng_review.index, eng_review.values)):
    game_name = f"{game_name[0]}_{game_name[1]}"
    game_review = preprocessing(game_review, 'lemma')
    game_review = [j for i in game_review for j in set(i)]
    game_review = [i for i in game_review if len(i) > 2]
    dict_eng[game_name] = game_review

10it [00:09,  1.04it/s]


In [ ]:
# 밑은 한국어 처리 내용

In [36]:
import numpy as np

In [186]:
tmp =  eng_review['game'].value_counts() > 250
games = []
for k,v in zip(tmp.index, tmp.values):
    if not v:
        break
    games.append(k)

In [7]:
total_game.remove('Gunfire Reborn')
total_game.remove('Amazing Frog?')

In [188]:
len(games)

654

In [189]:
ko_games = pickle.load( open('ko_games', 'rb'))

In [190]:
total_game = set(ko_games) & set(games)

In [3]:
total_game = pickle.load( open('total_game', 'rb'))

In [118]:
import pickle

pickle.dump(total_game, open('total_game', 'wb'))

In [117]:
len(total_game)

344

In [2]:
dict_kor = pickle.load(open('dict_kor', 'rb'))

In [135]:
count = Counter(dict_eng['Human: Fall Flat_0'])
word_list = count.most_common(100)

In [162]:
word_list

[('support', 17),
 ('linux', 10),
 ('time', 7),
 ('decision', 6),
 ('version', 6),
 ('review', 6),
 ('friend', 6),
 ('developer', 6),
 ('user', 5),
 ('platform', 5),
 ('hour', 5),
 ('camera', 5),
 ('control', 5),
 ('recommend', 5),
 ('window', 4),
 ('move', 4),
 ('devs', 4),
 ('publisher', 4),
 ('issue', 4),
 ('motion', 4),
 ('work', 4),
 ('money', 4),
 ('customer', 4),
 ('level', 4),
 ('feature', 4),
 ('price', 4),
 ('something', 3),
 ('option', 3),
 ('sickness', 3),
 ('party', 3),
 ('dev', 3),
 ('multiplayer', 3),
 ('anything', 3),
 ('content', 3),
 ('way', 3),
 ('year', 3),
 ('fall', 3),
 ('download', 2),
 ('force', 2),
 ('favor', 2),
 ('release', 2),
 ('comment', 2),
 ('doe', 2),
 ('scummy', 2),
 ('everyone', 2),
 ('product', 2),
 ('solo', 2),
 ('change', 2),
 ('cause', 2),
 ('movement', 2),
 ('edit', 2),
 ('patch', 2),
 ('character', 2),
 ('show', 2),
 ('fact', 2),
 ('situation', 2),
 ('face', 2),
 ('task', 2),
 ('http', 2),
 ('steamcommunity', 2),
 ('apology', 2),
 ('thank', 2),


In [161]:
count = Counter(dict_eng['Human: Fall Flat_0'])
word_list = count.most_common(100)

In [154]:
count = Counter(dict_kor['Human: Fall Flat'])
word_list = count.most_common(100)

NameError: name 'dict_kor' is not defined

In [12]:
game_list[0].items()

dict_items([('100% Orange Juice', (['market', 'file', 'friend', 'side', 'fake', 'moderation', 'bounty', 'effect', 'qhgrw', 'lash', 'drama', 'trigger', 'issue', 'work', 'retention', 'pet', 'power', 'victim', 'card', 'function', 'style', 'staff', 'design', 'amount', 'rng', 'com', 'price', 'matter', 'behaviour', 'hunt', 'balance', 'check', 'school', 'dot', 'stuff', 'item', 'grinder', 'system', 'forum', 'name', 'estimate', 'doe', 'quality', 'bloat', 'poppo', 'ozl', 'kid', 'png', 'case', 'sign', 'community', 'playtime', 'anyone', 'mio', 'orange', 'sora', 'toxic', 'life', 'stranger', 'money', 'hidden', 'gacha', 'txt', 'version', 'grind', 'dislike', 'truth', 'character', 'evidence', 'trading', 'board', 'hell', 'tldr', 'remain', 'focus', 'plain', 'criticism', 'misuse', 'refuse', 'clue', 'post', 'speak', 'rule', 'review', 'abuse', 'lie', 'pure', 'nothing', 'incompetent', 'man', 'pack', 'mascot', 'advantadge', 'weakness', 'lack', 'mind', 'jkukvan', 'use', 'grindstone', 'player', 'way', 'ban', 's

In [16]:
count_dict_eng = {}
for i in range(len(game_list)):
    for k,v in game_list[i].items():
        v = v[0] + v[1]
        count_dict_eng[k] =  Counter(v).most_common(100)

In [3]:
dict_kor.items()

dict_items([('EVE Online_0', ['실행', '직접', '연구', '어이', '하나', '매너', '게임', '인터넷', '미사일', '뉴비', '게임', '뉴비', '인격', '모독', '종류', '질문', '뉴비', '서로', '생각', '자기', '처음', '인물', '해외', '서버', '기분', '배척', '처음', '채팅', '번역', '기존', '유저', '사냥', '탐사', '재미', '전제', '보고', '단점', '트리', '마음', '내면', '현생', '장비', '탐사', '상대방', '웜홀', '심화', '시킴', '기본', '탐험', '플레이', '때문', '처음', '프타', '모두', '회사', '생활', '고여', '핑프', '하나', '인생', '탐사', '상임', '전쟁', '절대', '고급', '스킬', '함선', '게임', '함선', '무료', '채굴', '거나', '인물', '스캔', '좌표', '저장', '히스', '꼬접', '하드', '모드', '공부', '신경', '커서', '추천', '대상', '꼬와', '결제', '기초', '그것', '공격', '장점', '게임', '이기', '구입', '뉴비', '존나', '우주', '시간', '뉴비', '적대', '적임', '안내', '대기', '다음', '구조', '격차', '무료', '캡슐', '트리', '유료', '아이템', '누가', '게임', '진입', '장벽', '일단', '리뷰', '병신', '튜토리얼', '지랄', '뭐함', '인물', '검색', '새끼', '갑질', '새끼', '현실', '무시', '텃세', '새끼', '오지', '헤메', '병신', '새끼', '그거', '계속', '게임', '온갖', '소리', '한국인', '끼리', '쓰레기', '소리', '디씨', '스타일', '게임', '벌레', '먼저', '해대', '어투', '처음', '가격', '뜬금', '오메가', '인상', '망겜임', '객지', '간주', '전혀', '결제'

In [129]:
count_dict_kor = {}
for k,v in dict_kor.items():
    count_dict_kor[k] =  Counter(v).most_common(100)

In [105]:
count_dict_eng = {}
for k,v in dict_eng.items():
    count_dict_eng[k] =  Counter(v).most_common(100)

In [104]:
from collections import Counter

In [17]:
#total_word_kor = [[j[0] for j in i] for i in count_dict_kor.values()]
#total_word_kor = [j for i in total_word_kor for j in i]
total_word_eng = [[j[0] for j in i] for i in count_dict_eng.values()]
total_word_eng = [j for i in total_word_eng for j in i]

In [18]:
Counter(total_word_eng).most_common(100)

[('time', 344),
 ('way', 344),
 ('hour', 344),
 ('review', 343),
 ('work', 343),
 ('something', 342),
 ('experience', 339),
 ('everything', 338),
 ('look', 337),
 ('player', 336),
 ('point', 335),
 ('feel', 335),
 ('nothing', 335),
 ('bit', 323),
 ('part', 318),
 ('end', 316),
 ('love', 312),
 ('year', 307),
 ('level', 306),
 ('day', 305),
 ('use', 303),
 ('anything', 299),
 ('price', 287),
 ('life', 285),
 ('money', 275),
 ('system', 275),
 ('problem', 275),
 ('doe', 270),
 ('issue', 268),
 ('start', 263),
 ('story', 258),
 ('try', 257),
 ('reason', 257),
 ('character', 252),
 ('amount', 249),
 ('kind', 247),
 ('recommend', 247),
 ('world', 245),
 ('change', 236),
 ('need', 223),
 ('everyone', 211),
 ('bug', 204),
 ('content', 202),
 ('sale', 200),
 ('friend', 198),
 ('place', 197),
 ('help', 196),
 ('option', 195),
 ('style', 189),
 ('map', 183),
 ('mode', 179),
 ('enemy', 177),
 ('music', 175),
 ('anyone', 173),
 ('control', 170),
 ('developer', 167),
 ('buy', 166),
 ('design', 165)

In [143]:
Counter(total_word).most_common(100)

[('게임', 10),
 ('생각', 10),
 ('플레이', 10),
 ('정도', 10),
 ('재미', 10),
 ('정말', 10),
 ('때문', 10),
 ('진짜', 10),
 ('시간', 9),
 ('그냥', 9),
 ('추천', 9),
 ('다시', 9),
 ('다른', 9),
 ('시작', 8),
 ('사람', 8),
 ('처음', 8),
 ('하나', 8),
 ('한번', 8),
 ('느낌', 8),
 ('부분', 8),
 ('자체', 8),
 ('조금', 7),
 ('그래픽', 7),
 ('친구', 7),
 ('평가', 7),
 ('유저', 6),
 ('보고', 6),
 ('별로', 6),
 ('혼자', 6),
 ('플레이어', 6),
 ('모든', 6),
 ('인생', 5),
 ('문제', 5),
 ('가지', 5),
 ('이해', 5),
 ('일단', 5),
 ('추가', 5),
 ('사실', 5),
 ('라면', 5),
 ('매우', 5),
 ('캐릭터', 5),
 ('엔딩', 5),
 ('스토리', 5),
 ('진행', 5),
 ('요소', 5),
 ('인물', 4),
 ('시스템', 4),
 ('컨텐츠', 4),
 ('계속', 4),
 ('무슨', 4),
 ('현실', 4),
 ('이상', 4),
 ('초반', 4),
 ('가격', 4),
 ('구매', 4),
 ('배경', 4),
 ('거의', 4),
 ('단점', 4),
 ('지금', 4),
 ('클리어', 4),
 ('도전', 4),
 ('난이도', 4),
 ('중간', 4),
 ('작품', 4),
 ('분위기', 4),
 ('이벤트', 4),
 ('후반', 4),
 ('스팀', 3),
 ('스킬', 3),
 ('이유', 3),
 ('당신', 3),
 ('영어', 3),
 ('한글화', 3),
 ('노잼', 3),
 ('설명', 3),
 ('한국', 3),
 ('모두', 3),
 ('삭제', 3),
 ('공격', 3),
 ('끼리', 3),
 ('거기', 3),
 ('방법',

In [117]:
dict_kor['EVE Online_0']

['실행',
 '직접',
 '연구',
 '어이',
 '하나',
 '매너',
 '게임',
 '인터넷',
 '미사일',
 '뉴비',
 '게임',
 '뉴비',
 '인격',
 '모독',
 '종류',
 '질문',
 '뉴비',
 '서로',
 '생각',
 '자기',
 '처음',
 '인물',
 '해외',
 '서버',
 '기분',
 '배척',
 '처음',
 '채팅',
 '번역',
 '기존',
 '유저',
 '사냥',
 '탐사',
 '재미',
 '전제',
 '보고',
 '단점',
 '트리',
 '마음',
 '내면',
 '현생',
 '장비',
 '탐사',
 '상대방',
 '웜홀',
 '심화',
 '시킴',
 '기본',
 '탐험',
 '플레이',
 '때문',
 '처음',
 '프타',
 '모두',
 '회사',
 '생활',
 '고여',
 '핑프',
 '하나',
 '인생',
 '탐사',
 '상임',
 '전쟁',
 '절대',
 '고급',
 '스킬',
 '함선',
 '게임',
 '함선',
 '무료',
 '채굴',
 '거나',
 '인물',
 '스캔',
 '좌표',
 '저장',
 '히스',
 '꼬접',
 '하드',
 '모드',
 '공부',
 '신경',
 '커서',
 '추천',
 '대상',
 '꼬와',
 '결제',
 '기초',
 '그것',
 '공격',
 '장점',
 '게임',
 '이기',
 '구입',
 '뉴비',
 '존나',
 '우주',
 '시간',
 '뉴비',
 '적대',
 '적임',
 '안내',
 '대기',
 '다음',
 '구조',
 '격차',
 '무료',
 '캡슐',
 '트리',
 '유료',
 '아이템',
 '누가',
 '게임',
 '진입',
 '장벽',
 '일단',
 '리뷰',
 '병신',
 '튜토리얼',
 '지랄',
 '뭐함',
 '인물',
 '검색',
 '새끼',
 '갑질',
 '새끼',
 '현실',
 '무시',
 '텃세',
 '새끼',
 '오지',
 '헤메',
 '병신',
 '새끼',
 '그거',
 '계속',
 '게임',
 '온갖',
 '소리',
 '한국인',
 '끼리',
 '쓰레기

In [115]:
dict_eng['EVE Online_0']

{'EVE Online_0': ['fly',
  'safe',
  'player',
  'protection',
  'simply',
  'doing',
  'several',
  'unfortunately',
  'personal',
  'label',
  'casually',
  'low',
  'literally',
  'invested',
  'option',
  'faction',
  'haul',
  'black',
  'day',
  'solo',
  'try',
  'community',
  'pvp',
  'these',
  'eve',
  'back',
  'due',
  'invite',
  'destroyed',
  'however',
  'strong',
  'gate-camping',
  'leave',
  'seen',
  'feel',
  'whatever',
  'went',
  'believe',
  'bit',
  'precious',
  'multiboxing',
  'another',
  'reason',
  'high',
  'lack',
  'issue',
  'review',
  'month',
  'suffer',
  'subscribing',
  'tried',
  'hard',
  'considered',
  'perfect',
  'quickly',
  'null',
  'see',
  'carebear',
  'think',
  'joining',
  'sec',
  'someone',
  'relevant',
  'single',
  'yet',
  'happen',
  'time',
  'safe',
  'couple',
  'exploration',
  'under',
  'did',
  'exposing',
  'course',
  'deplorable',
  'ish',
  'mine',
  'work',
  'ship',
  'sever',
  'supposedly',
  'omega',
  'vi